# Notebook used for determining downed Transmission Lines

In [1]:
from tensorflow import keras
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
import pandas
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM
from sklearn.preprocessing import MinMaxScaler

Preprocessing Functions

In [2]:
def binary_data(data):
    shape = data.shape
    r, c = shape[0], shape[1]
    for i in range(r):
        for j in range(c):
            data.iloc[i][j] = data.iloc[i][j] / float((1.4013 ** (-45)))
            if data.iloc[i][j] > 0:
                data.iloc[i][j] = 1
    return data

def output_dat(data):
    t1 = data[1]
    t2 = data[3]
    t3 = data[4]
    t4 = data[6]
    t5 = data[7]
    t6 = data[8]
    t7 = data[9]
    return t1, t2, t3, t4, t5, t6, t7

## Tensorflow Models For Detecting Outages. CNN (LeNet5) and MLP

In [3]:
# List of Potential Models
def cnn_1():
    dim = (94, 1)
    x_input = tf.keras.Input(shape=dim)
    x = Conv1D(94, kernel_size=2, padding='same', activation='relu')(x_input)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(24, kernel_size=2, padding='same', activation='relu')(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(64, kernel_size=2, padding='same', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(16, activation="softmax")(x)
    model = tf.keras.models.Model(inputs=[x_input], outputs=[x])
    model.compile(loss='categorical_crossentropy', optimizer="sgd", metrics=['accuracy'])
    return model

def cnn_2():
    dim = (94, 1)
    x_input = tf.keras.Input(shape=dim)
    x = Conv1D(filters=32, kernel_size=3, activation='relu')(x_input)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(16, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=[x_input], outputs=[x])
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

def cnn_3():
    dim = (94, 1)
    x_input = tf.keras.Input(shape=dim)
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x_input)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(16, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=[x_input], outputs=[x])
#     opt = tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model    
    
def mlp():
    dim = (94)
    x_input = tf.keras.Input(shape=dim)
    x = Dense(100, activation='relu')(x_input)
    x = Dense(50, activation='relu')(x)
    x = Dense(25, activation='relu')(x)
#     x = Dense(25, activation='relu')(x)
    x = Dense(16)(x)
    model = tf.keras.models.Model(inputs=[x_input], outputs=[x])
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model
    
def mlp2(): # Test Purposes
    dim = (94)
    x_input = tf.keras.Input(shape=dim)
    x = Dense(100, activation='relu')(x_input)
    x = Dense(50, activation='relu')(x)
    x = Dense(25, activation='relu')(x)
    
    x1= Dense(1, name='Line1', activation='sigmoid')(x)
    x2= Dense(1, name='Line2', activation='sigmoid')(x)
    x3= Dense(1, name='Line3', activation='sigmoid')(x)
    x4= Dense(1, name='Line4', activation='sigmoid')(x)
    x5= Dense(1, name='Line5', activation='sigmoid')(x)
    x6= Dense(1, name='Line6', activation='sigmoid')(x)
    x7= Dense(1, name='Line7', activation='sigmoid')(x)
    
    opt = tf.keras.optimizers.Adam(lr=0.001)

    model = tf.keras.models.Model(inputs=[x_input], outputs=[x1,x2,x3,x4,x5,x6,x7])
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def cnn_4(): # Testing LeNet5 Model, Parameters can be tweeked
    dim = (94, 1)
    x_input = tf.keras.Input(shape=dim)
    x = Conv1D(94, kernel_size=2, padding='same', activation='relu')(x_input)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(24, kernel_size=2, padding='same', activation='relu')(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Conv1D(64, kernel_size=2, padding='same', activation='relu')(x)
    x = Flatten()(x)
    
    x1= Dense(1, activation='sigmoid', name='Line1')(x)
    x2= Dense(1, activation='sigmoid', name='Line2')(x)
    x3= Dense(1, activation='sigmoid', name='Line3')(x)
    x4= Dense(1, activation='sigmoid', name='Line4')(x)
    x5= Dense(1, activation='sigmoid', name='Line5')(x)
    x6= Dense(1, activation='sigmoid', name='Line6')(x)
    x7= Dense(1, activation='sigmoid', name='Line7')(x)
    
    model = tf.keras.models.Model(inputs=[x_input], outputs=[x1,x2,x3,x4,x5,x6,x7])
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

## Preprocessing of Data

In [4]:
# Preprocess data correctly

scaler = MinMaxScaler()
train_input = pd.read_csv("train_input.csv",header=None).transpose()
test_input = pd.read_csv("test_input.csv", header=None).transpose()
train_input_c = scaler.fit_transform(train_input)
test_input_c = scaler.fit_transform(test_input)

train_output = pd.read_csv("train_output.csv",header=None)
train_output = binary_data(train_output)
train_output = train_output.transpose()
test_output = pd.read_csv("test_output_c.csv", header=None).transpose() # Data already put between 0,1

# List of data for 7 different features of model
tr_out = output_dat(train_output)
te_out = output_dat(test_output)

## Load and Fit Model

In [5]:
# model = cnn_1()
model = cnn_4()

# data = model.fit(train_input, train_output, validation_data=(test_input, test_output),  epochs=5)
data = model.fit(train_input_c, tr_out, validation_data=(test_input_c, te_out),  epochs=10) # 10 epochs is probably overkill

Epoch 1/10
253/253 [==============================] - 7s 26ms/step - loss: 1.7638 - Line1_loss: 0.2527 - Line2_loss: 0.2463 - Line3_loss: 0.2513 - Line4_loss: 0.2569 - Line5_loss: 0.2409 - Line6_loss: 0.2599 - Line7_loss: 0.2558 - Line1_accuracy: 0.9302 - Line2_accuracy: 0.9335 - Line3_accuracy: 0.9278 - Line4_accuracy: 0.9353 - Line5_accuracy: 0.9285 - Line6_accuracy: 0.9297 - Line7_accuracy: 0.9300 - val_loss: 0.9920 - val_Line1_loss: 0.1343 - val_Line2_loss: 0.1191 - val_Line3_loss: 0.1303 - val_Line4_loss: 0.1536 - val_Line5_loss: 0.1218 - val_Line6_loss: 0.1716 - val_Line7_loss: 0.1613 - val_Line1_accuracy: 0.9612 - val_Line2_accuracy: 0.9576 - val_Line3_accuracy: 0.9579 - val_Line4_accuracy: 0.9577 - val_Line5_accuracy: 0.9575 - val_Line6_accuracy: 0.9575 - val_Line7_accuracy: 0.9593
Epoch 2/10
253/253 [==============================] - 6s 24ms/step - loss: 0.4462 - Line1_loss: 0.0266 - Line2_loss: 0.0208 - Line3_loss: 0.0286 - Line4_loss: 0.0693 - Line5_loss: 0.0666 - Line6_loss

253/253 [==============================] - 7s 29ms/step - loss: 0.0051 - Line1_loss: 2.7211e-06 - Line2_loss: 7.0353e-04 - Line3_loss: 3.3475e-04 - Line4_loss: 0.0019 - Line5_loss: 2.7292e-04 - Line6_loss: 0.0017 - Line7_loss: 1.7422e-04 - Line1_accuracy: 1.0000 - Line2_accuracy: 0.9999 - Line3_accuracy: 0.9999 - Line4_accuracy: 0.9996 - Line5_accuracy: 0.9999 - Line6_accuracy: 0.9999 - Line7_accuracy: 1.0000 - val_loss: 0.0016 - val_Line1_loss: 1.7780e-06 - val_Line2_loss: 1.6217e-06 - val_Line3_loss: 4.5656e-06 - val_Line4_loss: 5.3331e-05 - val_Line5_loss: 1.2785e-04 - val_Line6_loss: 0.0013 - val_Line7_loss: 1.1169e-04 - val_Line1_accuracy: 1.0000 - val_Line2_accuracy: 1.0000 - val_Line3_accuracy: 1.0000 - val_Line4_accuracy: 1.0000 - val_Line5_accuracy: 1.0000 - val_Line6_accuracy: 0.9997 - val_Line7_accuracy: 1.0000


In [6]:
# Save the Model
# model.save('Model_3_LeNet5_4_multiout')

In [7]:
# del model
# tf.keras.backend.clear_session()

In [8]:
# model = tf.keras.models.load_model("Model_3_LeNet5_4_multiout")

## Test the Model on the Test Data (for one set of inputs)

In [9]:
# Seven Outputs Either 1 or 0, should output [0, 1, 1, 1, 1, 1, 1]
x = tf.constant(test_input_c[1], dtype = tf.float32, shape=[1, 94])
y =model.predict(x)
y

[array([[8.846425e-08]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.99999034]], dtype=float32),
 array([[0.99992967]], dtype=float32),
 array([[1.]], dtype=float32)]